___
# mu7RON Demo 1
---

MuGen contains some useful tools for quickly analyzing and manipulating midi.

We will go over some of the most useful tools in this notebook.

---
### Midi Playback
---
MuGen is designed to be used alongside the [python3-midi](https://github.com/louisabraham/python3-midi/tree/louisabraham-patch-1) package. `python3-midi` allows for loading and saving operations as well as an API for interacting with the midi standard in Python.

In [1]:
import midi # python-midi

from mu7ron import utils 

pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html


As can be seen above, MuGen uses pygame as a backend for midi playback functionality.

We can load up a midi file using the python3-midi package:

In [2]:
fname = r'mu7ron/data/midi/demo/test0.mid'

ptrn = midi.read_midifile(fname)

type(ptrn)

midi.containers.Pattern

Mugen provides a useful api for midi playback of midi Patterns

In [3]:
utils.play(ptrn, 3.5)

The second argument, '3.5' defined how long the playback should last.

midi.Pattern objects often contain multiple midi.Track objects which may contain the musical information for different instruments in a piece of music.

For example:

In [4]:
print(f'ptrn contains {len(ptrn)} midi.Track objects')
ptrn[0], ptrn[1][:5]

ptrn contains 7 midi.Track objects


(midi.Track(\
   [midi.SetTempoEvent(tick=0, data=[6, 102, 252]),
    midi.TrackNameEvent(tick=0, text='', data=[]),
    midi.TimeSignatureEvent(tick=0, data=[4, 2, 6, 102]),
    midi.EndOfTrackEvent(tick=0, data=[])]),
 midi.Track(\
   [midi.TrackNameEvent(tick=0, text='C', data=[67]),
    midi.ControlChangeEvent(tick=0, channel=0, data=[100, 0]),
    midi.ControlChangeEvent(tick=0, channel=0, data=[101, 0]),
    midi.ControlChangeEvent(tick=0, channel=0, data=[6, 12]),
    midi.PitchWheelEvent(tick=0, channel=0, data=[0, 64])]))

midi.Track objects are containers for midi.events objects (as above). All together, these are used to tell the computer which pitches should be played at what times in order to construct the piece of music each time it is played.

The playback function may also be applied to a Track object:

In [5]:
trck = ptrn[1]
print(type(trck))
utils.play(trck)

<class 'midi.containers.Track'>


It should be noted that not every track will contain midi.NoteOnEvents or midi.NoteOffEvents. Some may just contain meta information. So doing this may not always produce music.

You can also drop in the path to the file...

In [6]:
utils.play(fname)

---
### MidiObj
---

Another way to play midi files/objects is with the MidiObj class:

In [7]:
midi_obj = utils.MidiObj(fname)

midi_obj.play(3.5)

The idea of the MidiObj class was to make it easy to quickly visualize some of the attributes of a piece of midi music. It can also take Pattern or Track objects as input when creating an instance.

i.e. time signature, tempo and instrument

For example:

In [8]:
midi_obj

              
        __________________________________________________________________________
        
        File          : test0.mid
        Location      : W:\OneDrive\__Dev__\__Dev__\AI Music\MusicEvo Project\data\midi\temp\working
        Repr          : <MuGen.utils.MidiObj object at 0x1a228f77888>
        Resolution    : 480
        ---------------------------------------------------------------------------
        
        Voices        :
            n. voice  : 1
            n. u_inst : 1
            data.     : i  | Group     | Instrument
                        -------------------------
                        75 | Pipe      | Recorder
                        75 | Pipe      | Recorder
                        75 | Pipe      | Recorder
                        75 | Pipe      | Recorder
                        75 | Pipe      | Recorder
                        75 | Pipe      | Recorder
            u. trck   : [False, True, True, True, True, True, True]

        Time sig.   

A MidiObj's midi.Pattern may be accessed by:

instance_name.ptrn

In [9]:
type(midi_obj.ptrn)

midi.containers.Pattern

The edit module contains useful tools for manipulating midi:

In [10]:
import MuGen.edit as edit

We can strip out a lot of event types untill we have the 'bare bones':

In [11]:
typs_2_keep = (
    midi.NoteOnEvent,
    midi.NoteOffEvent,
    midi.SetTempoEvent,
)

midi_obj.ptrn = edit.filter_ptrn_of_evnt_typs(midi_obj.ptrn, typs_2_keep=typs_2_keep)

It can be heard that the midi standard uses piano as a default instrument, after we have stripped all of the instrument information:

In [12]:
midi_obj.play(3.5)

Another very useful tool is the ability to consolidate tracks:

In [13]:
print(f'no. of tracks before: {len(midi_obj.ptrn)}')
midi_obj.ptrn = edit.consolidate_trcks(midi_obj.ptrn)
print(f'no. of tracks after: {len(midi_obj.ptrn)}')

no. of tracks before: 7
no. of tracks after: 1


As can be heard below, the music is preserved:

In [14]:
midi_obj.play(3.5)

Save progress with:

In [16]:
where_to_save = r'MuGen/data/midi/demo/result0.mid'

midi_obj.save(where_to_save)

Although, it should be noted that stripping certain events and meta information as we have may make the resulting midi file unable to be played back in some players. 

---
### Event Generator
---

The Mugen.utils module also contains a useful function that will return a generator that iterates over a midi.Track/midi.Pattern/MidiObj or any other common iterable, yielding a midi.events object each time:

In [17]:
gen = utils.evnt_gen(midi_obj)

In [18]:
for _ in range(10):
    print(gen.__next__())

midi.SetTempoEvent(tick=0, data=[6, 102, 252])
midi.NoteOnEvent(tick=0, channel=0, data=[84, 79])
midi.NoteOnEvent(tick=0, channel=1, data=[81, 79])
midi.NoteOnEvent(tick=0, channel=2, data=[77, 79])
midi.NoteOnEvent(tick=0, channel=3, data=[69, 79])
midi.NoteOnEvent(tick=0, channel=4, data=[60, 79])
midi.NoteOnEvent(tick=0, channel=5, data=[53, 79])
midi.NoteOffEvent(tick=390, channel=1, data=[81, 80])
midi.NoteOnEvent(tick=90, channel=1, data=[81, 79])
midi.NoteOffEvent(tick=90, channel=0, data=[84, 80])


In [19]:
type(gen), len(list(gen))

(generator, 161)